In [1]:
import theano

In [2]:
import os, sys
sys.path.insert(1, os.path.join('utils'))

from __future__ import print_function, division

path = 'data/statefarm/'
import utils; reload(utils)
from utils import *

Using Theano backend.


In [3]:
batch_size=16
vgg = Vgg16()
model = vgg.model
last_conv_idx = [i for i, l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx + 1]
conv_model = Sequential(conv_layers)

In [4]:
gen = image.ImageDataGenerator()
test_batches = get_batches(path + 'test', batch_size=batch_size, shuffle=False)

Found 79726 images belonging to 1 classes.


In [5]:
# conv_test_feat = conv_model.predict_generator(test_batches, test_batches.nb_sample)

In [7]:
fname = path + 'results/conv_feat_test.dat'
%rm -r $fname
for i in xrange(test_batches.n // batch_size):
    conv_feat = conv_model.predict_on_batch(test_batches.next()[0])
    if not i:
        c = bcolz.carray(conv_feat, rootdir= path + '/results/conv_test_feat.dat', mode='a')
    else:
        c.append(conv_feat)
c.flush()

rm: cannot remove 'data/statefarm/results/conv_feat_test.dat': No such file or directory


In [ ]:
conv_test_feat_batches = bcolz.iterblocks(path + fname, blen=batch_size)

In [ ]:
conv_train_feat_batches = get_batches(path + '/results/conv_feat.dat')
conv_valid_feat_batches = get_batches(path + '/results/conv_val_feat.dat')

In [ ]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

In [ ]:
p = 0.8
bn_model = Sequential([
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p/2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ])
bn_model.compile(Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Sequential.fit_generator(self, generator, samples_per_epoch, nb_epoch, verbose=1, callbacks=None, validation_data=None, nb_val_samples=None, class_weight=None, max_q_size=10, nb_worker=1, pickle_safe=False, initial_epoch=0, **kwargs)
bn_model.fit_generator((conv_train_feat_batches, trn_labels), conv_train_feat_batches.nb_sample, nb_epoch=1,
                       validation_data=(conv_valid_feat_batches, val_labels), nb_val_samples=conv_valid_feat_batches.nb_sample)

In [4]:
# ??Sequential.fit_generator
# ??Sequential.fit